In [2]:
import torch
from src.transformers.models.mamba.configuration_mamba import MambaConfig
from src.transformers.models.mamba.modeling_mamba import MambaMixer

/Users/alexandretorres/miniconda3/envs/torch23/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.manual_seed(23456)
config1 = MambaConfig(use_mambapy=False, vocab_size=60, hidden_size=64, num_hidden_layers=4)
mixer1 = MambaMixer(config1, 0)

torch.manual_seed(23456)
config2 = MambaConfig(use_mambapy=True, vocab_size=60, hidden_size=64, num_hidden_layers=4)
mixer2 = MambaMixer(config2, 0)

The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. Falling back to the naive implementation. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d


In [4]:
x = torch.randn(2, 12, 64)

In [5]:
torch.allclose(mixer1(x), mixer2(x))

True

In [ ]:
# TODO : CHECK BACKWARD!!!!!!